# HISTORY

#### Context

* This dataset was originally for a machine learning challenge to classify heart beat sounds. The data was gathered from two sources: (A) from the general public via the iStethoscope Pro iPhone app, and (B) from a clinic trial in hospitals using the digital stethoscope DigiScope. There were two challenges associated with this competition:

1. Heart Sound Segmentation

The first challenge is to produce a method that can locate S1(lub) and S2(dub) sounds within audio data, segmenting the Normal audio files in both datasets.

2. Heart Sound Classification

The task is to produce a method that can classify real heart audio (also known as “beat classification”) into one of four categories.

#### Content

* The dataset is split into two sources, A and B:

* set_a.csv - Labels and metadata for heart beats collected from the general public via an iPhone app

* setatiming.csv - contains gold-standard timing information for the "normal" recordings from Set A.

* set_b.csv - Labels and metadata for heart beats collected from a clinical trial in hospitals using a digital stethoscope

* audio files - Varying lengths, between 1 second and 30 seconds. (some have been clipped to reduce excessive noise and provide the salient fragment of the sound)

# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
import imageio
from IPython.display import Image
import matplotlib.image as mpimg
#MUSIC PROCESS
import pydub
from scipy.io.wavfile import read, write
import librosa
import librosa.display
import IPython
from IPython.display import Audio
import scipy
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape,\
Conv2DTranspose, LeakyReLU, Conv1D, AveragePooling1D, MaxPooling1D, MaxPool1D, GlobalAvgPool1D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
from tensorflow.keras import regularizers
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PATH, LABEL, TRANSFORMATION

#### MAIN CSV

In [ ]:
Set_A_Data = pd.read_csv("../input/heartbeat-sounds/set_a.csv")
Set_B_Data = pd.read_csv("../input/heartbeat-sounds/set_b.csv")

In [ ]:
print(Set_A_Data.head(-1))

In [ ]:
print(Set_A_Data.isnull().sum())

In [ ]:
print(Set_A_Data["label"].value_counts())

In [ ]:
print(Set_A_Data["dataset"].value_counts())

In [ ]:
print(Set_B_Data.head(-1))

In [ ]:
print(Set_B_Data.isnull().sum())

In [ ]:
print(Set_B_Data["label"].value_counts())

In [ ]:
print(Set_B_Data["dataset"].value_counts())

#### TO CONCAT

In [ ]:
DataFrames = [Set_A_Data,Set_B_Data]
Concat_Data = pd.concat(DataFrames)

In [ ]:
print(Concat_Data.head(-1))

In [ ]:
print(Concat_Data.isnull().sum())

In [ ]:
Concat_Data.drop(["sublabel","dataset"],axis="columns",inplace=True)

In [ ]:
print(Concat_Data.head(-1))

In [ ]:
print(Concat_Data["label"].value_counts())

In [ ]:
Concat_Data = Concat_Data.dropna()

In [ ]:
print(Concat_Data.head(600))

In [ ]:
Concat_Data = Concat_Data.reset_index()

In [ ]:
print(Concat_Data.head(-1))

In [ ]:
print(Concat_Data.isnull().sum())

In [ ]:
Concat_Data.drop("index",axis="columns",inplace=True)

#### TO CREATE NEW DATAFRAME

In [ ]:
Path_Wav_List = []
Category_List = []

for path_number in range(585):
    File_Path_Name = "../input/heartbeat-sounds/" + str(Concat_Data["fname"][path_number])
    Path_Wav_List.append(File_Path_Name)
    Category_List.append(Concat_Data["label"][path_number])

In [ ]:
print(Path_Wav_List[0:4])

In [ ]:
print(Category_List[0:4])

In [ ]:
Path_Wav_Series = pd.Series(Path_Wav_List,name="WAV").astype(str)
Category_Series = pd.Series(Category_List,name="CATEGORY")

In [ ]:
Main_Heartbeat_Data = pd.concat([Path_Wav_Series,Category_Series],axis=1)

In [ ]:
print(Main_Heartbeat_Data.head(-1))

#### TO SHUFFLE

In [ ]:
Main_Heartbeat_Data = Main_Heartbeat_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
print(Main_Heartbeat_Data["CATEGORY"].value_counts())

In [ ]:
print(Main_Heartbeat_Data.head(-1))

#### TO SAVE

In [ ]:
Concat_Data.to_csv("New_Heartbeat_Doc.csv")

In [ ]:
Main_Heartbeat_Data.to_csv("Main_Heartbeat_Doc.csv")

# PROCESS FUNCTIONS

#### NOISE

In [ ]:
def noise_function(data):
    noise_value = 0.009 * np.random.uniform() * np.amax(data)
    data = data + noise_value * np.random.normal(size=data.shape[0])
    
    return data

#### STRETCH FUNCTION

In [ ]:
def stretch_function(data,rate=0.6):
    
    return librosa.effects.time_stretch(data,rate)

#### SHIFT FUNCTION

In [ ]:
def shift_function(data):
    
    shift_range = int(np.random.uniform(-3,3) * 1000)
    return np.roll(data,shift_range)

#### PITCH FUNCTION

In [ ]:
def pitch_function(data,sampling_rate,pitch_factor=0.3):
    
    return librosa.effects.pitch_shift(data,sampling_rate,pitch_factor)

#### SPECSHOW FUNCTION

In [ ]:
def specshow_function(wav_path):
    
    figure,axis = plt.subplots(4,1,figsize=(14,6))
    
    audio_type,sample_rate = librosa.load(wav_path)
    stft_audio = librosa.stft(audio_type)
    Db_audio = librosa.amplitude_to_db(abs(stft_audio))
    Img_Log = librosa.display.specshow(Db_audio,sr=sample_rate,x_axis="time",y_axis="log",cmap='gray_r',ax=axis[0])
    axis[0].set(title='LOG')
    Img_Mel = librosa.display.specshow(Db_audio,sr=sample_rate,x_axis="time",y_axis="mel",cmap='gray_r',ax=axis[1])
    axis[1].set(title='MEL')
    Img_Chroma = librosa.display.specshow(Db_audio,sr=sample_rate,x_axis="time",y_axis="chroma",cmap='gray_r',ax=axis[2])
    axis[2].set(title='CHROMA')
    Img_Hz = librosa.display.specshow(Db_audio,sr=sample_rate,x_axis="time",y_axis="hz",cmap='gray_r',ax=axis[3])
    axis[3].set(title='HZ')
    
    
    for ax_i in axis:
        ax_i.label_outer()

    
    figure.colorbar(Img_Log, ax=[axis[0], axis[1]])

    # Or have individual colorbars:
    figure.colorbar(Img_Chroma, ax=[axis[2],axis[3]])
    
    axis[0].set(xlim=[1, 3])

#### WAVEPLOT FUNCTION

In [ ]:
def waveplot_function(wav_path):
    figure = plt.figure(figsize=(14,6))
    
    audio_type,sample_rate = librosa.load(wav_path)
    librosa.display.waveplot(audio_type,sr=sample_rate)

#### PLAYING FUNCTION

In [ ]:
def playing_function(wav_path):
    
    audio_type,sample_rate = librosa.load(wav_path)
    return Audio(audio_type,rate=sample_rate)

#### EXTRACT FUNCTION

In [ ]:
def extract_function(data):
    
    output_result = np.array([])
    
    mean_zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=data).T,axis=0)
    output_result = np.hstack((output_result,mean_zero_crossing_rate))
    
    stft_output = np.abs(librosa.stft(data))
    chroma_mean = np.mean(librosa.feature.chroma_stft(S=stft_output,sr=sample_rate).T,axis=0)
    output_result = np.hstack((output_result,chroma_mean))
    
    mfcc_output = np.mean(librosa.feature.mfcc(y=data,sr=sample_rate).T,axis=0)
    output_result = np.hstack((output_result,mfcc_output))
    
    root_output = np.mean(librosa.feature.rms(y=data).T,axis=0)
    output_result = np.hstack((output_result,root_output))
    
    mel_output = np.mean(librosa.feature.melspectrogram(y=data,sr=sample_rate).T,axis=0)
    output_result = np.hstack((output_result,mel_output))
    
    return output_result

#### EXPORT FUNCTION

In [ ]:
def export_function(path):
    
    data,sample_rate = librosa.load(path,duration=3.0)
    
    Output_One = extract_function(data)
    result = np.array(Output_One)
    
    noise_output = noise_function(data)
    Output_Two = extract_function(noise_output)
    result = np.vstack((result,Output_Two))
    
    stretch_output = stretch_function(data)
    stretch_pitch = pitch_function(stretch_output,sample_rate)
    Output_Three = extract_function(stretch_pitch)
    result = np.vstack((result,Output_Three))
    
    return result

# ANALYSIS

In [ ]:
plt.style.use("dark_background")

#### PLAYING

In [ ]:
try:
    playing_function(Main_Heartbeat_Data["WAV"][580])
except Exception as error:
    print("NO DIRECTORY ERROR")

In [ ]:
playing_function(Main_Heartbeat_Data["WAV"][480])

#### WAVEPLOT

In [ ]:
waveplot_function(Main_Heartbeat_Data["WAV"][480])

#### SPECSHOW

In [ ]:
specshow_function(Main_Heartbeat_Data["WAV"][480])

#### NOISE

In [ ]:
wav_type,sample_rate = librosa.load(Main_Heartbeat_Data["WAV"][480])
noise_audio = noise_function(wav_type)
Audio(noise_audio,rate=sample_rate)

#### STRETCH

In [ ]:
wav_type,sample_rate = librosa.load(Main_Heartbeat_Data["WAV"][480])
stretch_audio = stretch_function(wav_type)
Audio(stretch_audio,rate=sample_rate)

#### SHIFT

In [ ]:
wav_type,sample_rate = librosa.load(Main_Heartbeat_Data["WAV"][480])
shift_audio = shift_function(wav_type)
Audio(shift_audio,rate=sample_rate)

#### PITCH

In [ ]:
wav_type,sample_rate = librosa.load(Main_Heartbeat_Data["WAV"][480])
pitch_audio = pitch_function(wav_type,sample_rate)
Audio(pitch_audio,rate=sample_rate)

#### SIMPLE CHECKING

In [ ]:
wav_type,sample_rate = librosa.load(Main_Heartbeat_Data["WAV"][480])

print(wav_type.shape)
print(wav_type.dtype)
print(sample_rate)

# DATA PROCESS AND ENGINEERING

In [ ]:
sample_rate = 22050

In [ ]:
x_Train = []
y_Train = []
    
for path,category_wav in zip(Main_Heartbeat_Data.WAV,Main_Heartbeat_Data.CATEGORY):
    
    try:  
        wav_features = export_function(path)
    
        for indexing in wav_features:
            x_Train.append(indexing)
            y_Train.append(category_wav)

    except Exception as e:
        print("NO DIRECTORY ERROR: ", path)

In [ ]:
New_Heartbeat_Wav = pd.DataFrame(x_Train)
New_Heartbeat_Wav["CATEGORY"] = y_Train

New_Heartbeat_Wav.to_csv("New_Wav_Heartbeat_Data.csv",index=False)

In [ ]:
New_Heartbeat_Wav.head(-1)

In [ ]:
print(New_Heartbeat_Wav["CATEGORY"].value_counts())

#### SPLITTING AND PRE-TRAINING PROCESS

In [ ]:
Encoder_Function = OneHotEncoder()
Scaler_Function = MinMaxScaler()

In [ ]:
Values_X = New_Heartbeat_Wav.iloc[:,:-1].values
Labels_X = New_Heartbeat_Wav["CATEGORY"].values

In [ ]:
print(Values_X.shape)
print(Labels_X.shape)

In [ ]:
Labels_X_Encode = Encoder_Function.fit_transform(np.array(Labels_X).reshape(-1,1)).toarray()

In [ ]:
print(Labels_X_Encode[0:5])

In [ ]:
print(Labels_X_Encode.shape)

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(Values_X,Labels_X_Encode,train_size=0.9,random_state=42,shuffle=True)

In [ ]:
print(xTrain.shape)
print(yTrain.shape)
print(xTest.shape)
print(yTest.shape)

In [ ]:
xTrain = np.expand_dims(xTrain,axis=2)
xTest = np.expand_dims(xTest,axis=2)

In [ ]:
print(xTrain.shape)
print(xTest.shape)

# MODEL STRUCTURE

#### PARAMETERS

In [ ]:
output_dim = 4
compile_metrics = ["accuracy"]
compile_loss = "categorical_crossentropy"
compile_optimizer = Adam()
input_shape_dim = (xTrain.shape[1],1)

#### CALLBACKS

In [ ]:
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")

#### STRUCTURE

In [ ]:
Model = Sequential()

Model.add(Conv1D(256,5,strides=1,padding="same",activation="relu",input_shape=input_shape_dim))
Model.add(BatchNormalization())
Model.add(MaxPooling1D(3,strides=2,padding="same"))

Model.add(Conv1D(256,4,strides=1,padding="same",activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling1D(3,strides=2,padding="same"))

Model.add(Conv1D(128,4,strides=1,padding="same",activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling1D(3,strides=2,padding="same"))


Model.add(Conv1D(64,4,strides=1,padding="same",activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling1D(3,strides=2,padding="same"))

Model.add(Conv1D(32,4,strides=1,padding="same",activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling1D(3,strides=2,padding="same"))


Model.add(Flatten())
Model.add(Dense(1024, activation='relu'))
Model.add(Dropout(0.3))

Model.add(Dense(units=output_dim, activation='softmax'))

In [ ]:
print(Model.summary())

In [ ]:
print(Model.layers)

In [ ]:
Model.compile(optimizer=compile_optimizer,loss=compile_loss,metrics=compile_metrics)

In [ ]:
Conv1D_Model = Model.fit(xTrain, yTrain,
                         batch_size=12,
                         epochs=70,
                                validation_data=(xTest, yTest), callbacks=[Early_Stopper,Checkpoint_Model])

In [ ]:
Model_Results = Model.evaluate(xTest,yTest)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.4f" % Model_Results[1])

# MODEL STRUCTURE WITH CONV2D

#### DATA PROCESS AND SPLITTING FOR CONV2D

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Values_X,Labels_X_Encode, train_size=0.9, random_state = 42, shuffle=True)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 18, 9, 1)
X_test = X_test.reshape(X_test.shape[0], 18, 9, 1)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

#### STRUCTURE

In [ ]:
Model_Check_Conv2D = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_Conv2D_model")

In [ ]:
Model_Conv2D = Sequential()
#
Model_Conv2D.add(Conv2D(64,(3, 3),padding="same",activation="relu",input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])))
Model_Conv2D.add(MaxPool2D(pool_size=(2, 2)))
#
Model_Conv2D.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
Model_Conv2D.add(Dropout(0.3))
Model_Conv2D.add(MaxPool2D(pool_size=(2, 2)))

Model_Conv2D.add(Conv2D(256, (3, 3), padding = "same", activation = "relu"))
Model_Conv2D.add(Dropout(0.3))
Model_Conv2D.add(MaxPool2D(pool_size=(2, 2)))
#
Model_Conv2D.add(Flatten())
Model_Conv2D.add(Dense(2048, activation = "relu"))
Model_Conv2D.add(Dropout(0.5))
Model_Conv2D.add(Dense(1024, activation = "relu"))
Model_Conv2D.add(Dropout(0.5))
Model_Conv2D.add(Dense(output_dim, activation = "softmax"))

In [ ]:
Model_Conv2D.compile(optimizer=compile_optimizer,loss=compile_loss,metrics=compile_metrics)

In [ ]:
Conv2D_Model = Model_Conv2D.fit(X_train, Y_train, batch_size=64, epochs=70,
                                validation_data=(X_test, Y_test), callbacks=[Early_Stopper,Model_Check_Conv2D])

In [ ]:
Model_Results_Conv2D = Model_Conv2D.evaluate(X_test,Y_test)
print("LOSS:  " + "%.4f" % Model_Results_Conv2D[0])
print("ACCURACY:  " + "%.4f" % Model_Results_Conv2D[1])

# MODEL STRUCTURE WITH RNN - SEPCNN

In [ ]:
Model_Check_RNN_SEPCNN = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_RNN_SEPCNN_model")

In [ ]:
Model_R = Sequential()

Model_R.add(Conv2D(32,(3,3),activation="relu",
                 input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])))
Model_R.add(BatchNormalization())
Model_R.add(MaxPooling2D((2,2)))

#
Model_R.add(Conv2D(64,(3,3),
                 activation="relu",padding="same"))
Model_R.add(Dropout(0.2))
Model_R.add(MaxPooling2D((2,2)))


#
Model_R.add(TimeDistributed(Flatten()))
Model_R.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))
Model_R.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))

#
Model_R.add(Flatten())
Model_R.add(Dense(128,activation="relu"))
Model_R.add(Dropout(0.5))
Model_R.add(Dense(output_dim,activation="softmax"))

In [ ]:
Model_R.compile(optimizer=compile_optimizer,loss=compile_loss,metrics=compile_metrics)

In [ ]:
RNN_SEPCNN_Model = Model_R.fit(X_train, Y_train, epochs=70,
                                validation_data=(X_test, Y_test), callbacks=[Early_Stopper,Model_Check_RNN_SEPCNN])

In [ ]:
Model_Results_RNN_SEPCNN = Model_R.evaluate(X_test,Y_test)
print("LOSS:  " + "%.4f" % Model_Results_RNN_SEPCNN[0])
print("ACCURACY:  " + "%.4f" % Model_Results_RNN_SEPCNN[1])

# MODEL STRUCTURE WITH LSTM

#### DATA PROCESS AND SPLITTING FOR LSTM

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Values_X,Labels_X_Encode, train_size=0.9, random_state = 42, shuffle=True)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### STRUCTURE

In [ ]:
Model_Check_LSTM = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_LSTM_model")

In [ ]:
Model_LSTM = Sequential()

Model_LSTM.add(Bidirectional(LSTM(units=64,
                                  dropout=0.2,
                                  return_sequences=True),
                             input_shape=(X_train.shape[1],X_train.shape[2])))

Model_LSTM.add(Bidirectional(LSTM(units=32,
                                  dropout=0.2,
                                  return_sequences=False)))

Model_LSTM.add(Dense(output_dim, activation='softmax'))

In [ ]:
Model_LSTM.compile(optimizer="rmsprop",loss=compile_loss,metrics=compile_metrics)

In [ ]:
LSTM_Model = Model_LSTM.fit(X_train, Y_train, epochs=70,
                                validation_data=(X_test, Y_test), callbacks=[Early_Stopper,Model_Check_LSTM])

In [ ]:
Model_Results_LSTM = Model_LSTM.evaluate(X_test,Y_test)
print("LOSS:  " + "%.4f" % Model_Results_LSTM[0])
print("ACCURACY:  " + "%.4f" % Model_Results_LSTM[1])

# MODEL STRUCTURE WITH CON1D AND GLOBALAVGPOOL1D

#### DATA PROCESS AND SPLITTING FOR CON1D AND GLOBALAVGPOOL1D

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Values_X,Labels_X_Encode, train_size=0.9, random_state = 42, shuffle=True)

In [ ]:
X_train = np.expand_dims(X_train,axis=2)
X_test = np.expand_dims(X_test,axis=2)

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### STRUCTURE

In [ ]:
Model_Check_CONV1D_II = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_CONV1D_II_model")

In [ ]:
Model_CONV1D_II = Sequential()

Model_CONV1D_II.add(Conv1D(5,kernel_size=9,strides=1,padding="same",activation='relu',
                input_shape = (X_train.shape[1],1),
                kernel_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4)))
Model_CONV1D_II.add(BatchNormalization())
Model_CONV1D_II.add(MaxPooling1D(3,strides=2,padding="same"))

Model_CONV1D_II.add(Conv1D(15,kernel_size=9,strides=1,padding="same", activation='relu',
                kernel_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4)))
Model_CONV1D_II.add(Dropout(0.25))
Model_CONV1D_II.add(MaxPooling1D(3,strides=2,padding="same"))


Model_CONV1D_II.add(Conv1D(45,kernel_size=9,strides=1,padding="same", activation='relu',
                 kernel_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4)))
Model_CONV1D_II.add(Dropout(0.25))
Model_CONV1D_II.add(MaxPooling1D(3,strides=2,padding="same"))


Model_CONV1D_II.add(Conv1D(65,kernel_size=9,strides=1,padding="same", activation='relu'))
Model_CONV1D_II.add(Dropout(0.25))
Model_CONV1D_II.add(MaxPooling1D(3,strides=2,padding="same"))

Model_CONV1D_II.add(Conv1D(75,kernel_size=9,strides=1,padding="same", activation='relu'))
Model_CONV1D_II.add(Dropout(0.5))
Model_CONV1D_II.add(MaxPooling1D(3,strides=2,padding="same"))

Model_CONV1D_II.add(Conv1D(85,kernel_size=9,strides=1,padding="same", activation='relu'))
Model_CONV1D_II.add(Dropout(0.5))
Model_CONV1D_II.add(MaxPooling1D(3,strides=2,padding="same"))

Model_CONV1D_II.add(Dropout(0.75))
Model_CONV1D_II.add(GlobalAvgPool1D())
Model_CONV1D_II.add(Dense(output_dim, activation='softmax'))

In [ ]:
Model_CONV1D_II.compile(optimizer=compile_optimizer,loss=compile_loss,metrics=compile_metrics)

In [ ]:
CONV1D_II_Model = Model_CONV1D_II.fit(X_train, Y_train, epochs=70,
                                validation_data=(X_test, Y_test), callbacks=[Early_Stopper,Model_Check_CONV1D_II])

In [ ]:
Model_Results_CONV1D_II = Model_CONV1D_II.evaluate(X_test,Y_test)
print("LOSS:  " + "%.4f" % Model_Results_CONV1D_II[0])
print("ACCURACY:  " + "%.4f" % Model_Results_CONV1D_II[1])

# SKLEARN

#### DATA PROCESS AND SPLITTING FOR SKLEARN

In [ ]:
Label_Encode = LabelEncoder()

In [ ]:
Labels_X_Encode_II = Label_Encode.fit_transform(Labels_X)

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(Values_X,Labels_X_Encode_II,train_size=0.9,random_state=42,shuffle=True)

In [ ]:
print(xTrain.shape)
print(xTest.shape)
print(yTrain.shape)
print(yTest.shape)

In [ ]:
xTrain = Scaler_Function.fit_transform(xTrain)
xTest = Scaler_Function.fit_transform(xTest)

#### STRUCTURE

In [ ]:
lj = LogisticRegression(solver="liblinear").fit(xTrain,yTrain)
gnb = GaussianNB().fit(xTrain,yTrain)
knnc = KNeighborsClassifier().fit(xTrain,yTrain)
cartc = DecisionTreeClassifier(random_state=42).fit(xTrain,yTrain)
rfc = RandomForestClassifier(random_state=42,verbose=False).fit(xTrain,yTrain)
gbmc = GradientBoostingClassifier(verbose=False).fit(xTrain,yTrain)
xgbc = XGBClassifier().fit(xTrain,yTrain)
lgbmc = LGBMClassifier().fit(xTrain,yTrain)
catbc = CatBoostClassifier(verbose=False).fit(xTrain,yTrain)

In [ ]:
modelsc = [lj,gnb,knnc,cartc,rfc,gbmc,xgbc,lgbmc,catbc]
for model in modelsc:
    name = model.__class__.__name__
    predict = model.predict(xTest)
    R2CV = cross_val_score(model,xTest,yTest,cv=10,verbose=False).mean()
    error = -cross_val_score(model,xTest,yTest,cv=10,scoring="neg_mean_squared_error",verbose=False).mean()
    print(name + ": ")
    print("-" * 10)
    print("ACC-->",accuracy_score(yTest,predict))
    print("R2CV-->",R2CV)
    print("MEAN SQUARED ERROR-->",np.sqrt(error))
    print("-" * 30)